# XGBoost Baseline Model

This notebook implements an XGBoost model as a baseline for the transport footfall forecasting task.
It follows the same data preprocessing steps as the deep learning models to ensure a fair comparison.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings

# Configuration
SEED = 42
WINDOW_SIZE = 12
PREDICT_AHEAD = 1

# Reproducibility
np.random.seed(SEED)
warnings.filterwarnings('ignore')


In [2]:
# Paths
data_path = 'StationFootfall_Merged_2019-2023.csv'
coords_path = 'station_coords.csv'

# Load data
print("Loading data...")
df = pd.read_csv(data_path)

# Create Total Footfall
df['TotalFootfall'] = df['EntryTapCount'] + df['ExitTapCount']

# Pivot to wide format: Index=TravelDate, Columns=Station, Values=TotalFootfall
ts_data = df.pivot_table(index='TravelDate', columns='Station', values='TotalFootfall', aggfunc='sum')

# Convert index to datetime
ts_data.index = pd.to_datetime(ts_data.index)
ts_data = ts_data.sort_index()

# Load coordinates
station_coords = pd.read_csv(coords_path)
station_coords = station_coords.set_index('Station')
station_coords = station_coords[['latitude', 'longitude']]

# Align stations
common_stations = ts_data.columns.intersection(station_coords.index)
ts_data = ts_data[common_stations]
station_coords = station_coords.loc[common_stations]

# Replace 0 with NaN before resampling to avoid averaging zeros
ts_data = ts_data.replace(0, np.nan)

# Resample to weekly average
print("Resampling to weekly average...")
ts_data = ts_data.resample('W').mean()

print(f"Resampled data shape: {ts_data.shape}")


# 2. Supplement missing values
# Interpolate
ts_data = ts_data.interpolate(method='linear', limit_direction='both')
# Fill remaining NaNs with 0 (if any)
ts_data = ts_data.fillna(0)

print(f"Processed data shape: {ts_data.shape}")

Loading data...
Resampling to weekly average...
Resampled data shape: (261, 436)
Processed data shape: (261, 436)


In [3]:
# Feature Engineering for XGBoost
num_samples_total = len(ts_data) - WINDOW_SIZE - PREDICT_AHEAD + 1
train_samples_count = int(0.7 * num_samples_total)
# The last time step used in training (including target)
train_end_idx = train_samples_count + WINDOW_SIZE + PREDICT_AHEAD - 1

print(f"Fitting scaler on first {train_end_idx} time steps (Training set).")

scaler = StandardScaler()
footfall_cols = ts_data.columns # All columns are footfall now

# Fit on training data only
scaler.fit(ts_data.iloc[:train_end_idx][footfall_cols])

# Transform all data
ts_data[footfall_cols] = scaler.transform(ts_data[footfall_cols])

print("Data normalized (StandardScaler) - Fit on Train, Transformed All.")

def create_xgb_dataset(ts_data, window_size, predict_ahead):
    X_list = []
    y_list = []
    
    # stations = [c for c in ts_data.columns if c != 'WeekOfYear']
    stations = ts_data.columns
    
    # Iterate through time steps
    num_samples = len(ts_data) - window_size - predict_ahead + 1
    
    for idx in range(num_samples):
        # Extract window for all stations
        # Shape: (window_size, num_stations)
        window_data = ts_data.iloc[idx : idx + window_size]
        
        # Target time step
        target_idx = idx + window_size + predict_ahead - 1
        target_data = ts_data.iloc[target_idx]
        
        # WeekOfYear removed
        # target_week = target_data['WeekOfYear']
        
        # For each station, create a sample
        for station in stations:
            # Lag features: values of this station in the window
            lags = window_data[station].values
            
            # Features: [Lag_1, Lag_2, ..., Lag_12]
            # Removed target_week to match univariate DL baseline
            features = lags 
            
            target = target_data[station]
            
            X_list.append(features)
            y_list.append(target)
            
    return np.array(X_list), np.array(y_list)

print("Creating XGBoost dataset...")
X, y = create_xgb_dataset(ts_data, WINDOW_SIZE, PREDICT_AHEAD)
print(f"X shape: {X.shape}, y shape: {y.shape}")

Fitting scaler on first 186 time steps (Training set).
Data normalized (StandardScaler) - Fit on Train, Transformed All.
Creating XGBoost dataset...
X shape: (108564, 12), y shape: (108564,)


In [4]:
# Split Data into Training, Validation, and Test Sets

# Calculate number of time steps available for training/testing
num_time_steps = len(ts_data) - WINDOW_SIZE - PREDICT_AHEAD + 1
# num_stations = len([c for c in ts_data.columns if c != 'WeekOfYear'])
num_stations = len(ts_data.columns)

# Split indices based on time steps
train_steps = int(0.7 * num_time_steps)
val_steps = int(0.15 * num_time_steps)
test_steps = num_time_steps - train_steps - val_steps

print(f"Total time steps: {num_time_steps}")
print(f"Train steps: {train_steps}, Val steps: {val_steps}, Test steps: {test_steps}")

# The X and y arrays are ordered by time, then station.
# So the first (train_steps * num_stations) samples are training data.

train_size = train_steps * num_stations
val_size = val_steps * num_stations
test_size = test_steps * num_stations

X_train = X[:train_size]
y_train = y[:train_size]

X_val = X[train_size : train_size + val_size]
y_val = y[train_size : train_size + val_size]

X_test = X[train_size + val_size :]
y_test = y[train_size + val_size :]

print(f"Train shape: {X_train.shape}, {y_train.shape}")
print(f"Val shape: {X_val.shape}, {y_val.shape}")
print(f"Test shape: {X_test.shape}, {y_test.shape}")

Total time steps: 249
Train steps: 174, Val steps: 37, Test steps: 38
Train shape: (75864, 12), (75864,)
Val shape: (16132, 12), (16132,)
Test shape: (16568, 12), (16568,)


In [5]:
# Initialize and Train XGBoost Model

# Using simpler hyperparameters to establish a true "baseline" performance
# Reduced depth and estimators to avoid overfitting and simulate a standard baseline
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000, 
    learning_rate=0.1,    
    max_depth=6,         
    subsample=1.0,        
    colsample_bytree=1.0, 
    random_state=SEED,
    n_jobs=-1
)

print("Training XGBoost model...")
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=10
)

print("Training complete.")

Training XGBoost model...
[0]	validation_0-rmse:0.90042	validation_1-rmse:276.50733
[10]	validation_0-rmse:0.42000	validation_1-rmse:276.46907
[20]	validation_0-rmse:0.30234	validation_1-rmse:276.44976
[30]	validation_0-rmse:0.27605	validation_1-rmse:276.44391
[40]	validation_0-rmse:0.26579	validation_1-rmse:276.44173
[50]	validation_0-rmse:0.25887	validation_1-rmse:276.44079
[60]	validation_0-rmse:0.25429	validation_1-rmse:276.44044
[70]	validation_0-rmse:0.25091	validation_1-rmse:276.43972
[80]	validation_0-rmse:0.24738	validation_1-rmse:276.43913
[90]	validation_0-rmse:0.24482	validation_1-rmse:276.43889
[100]	validation_0-rmse:0.24184	validation_1-rmse:276.43871
[110]	validation_0-rmse:0.23885	validation_1-rmse:276.43868
[120]	validation_0-rmse:0.23600	validation_1-rmse:276.43870
[130]	validation_0-rmse:0.23307	validation_1-rmse:276.43874
[140]	validation_0-rmse:0.23041	validation_1-rmse:276.43886
[150]	validation_0-rmse:0.22823	validation_1-rmse:276.43879
[160]	validation_0-rmse:0

In [ ]:
# Evaluate Model Performance

# Predict on test set
y_pred = model.predict(X_test)

# Get scaler parameters
means = scaler.mean_
scales = scaler.scale_

# Create station indices for the test set
station_indices_test = np.tile(np.arange(num_stations), test_steps)

# Ensure shapes match (handle potential truncation if any)
min_len = min(len(y_test), len(y_pred), len(station_indices_test))
y_test_aligned = y_test[:min_len]
y_pred_aligned = y_pred[:min_len]
station_indices_test = station_indices_test[:min_len]

# Inverse transform (Flattened)
y_test_inv_flat = y_test_aligned * scales[station_indices_test] + means[station_indices_test]
y_pred_inv_flat = y_pred_aligned * scales[station_indices_test] + means[station_indices_test]

complete_steps = min_len // num_stations
truncate_idx = complete_steps * num_stations

y_test_reshaped = y_test_inv_flat[:truncate_idx].reshape(complete_steps, num_stations)
y_pred_reshaped = y_pred_inv_flat[:truncate_idx].reshape(complete_steps, num_stations)

print(f"Evaluated on {complete_steps} time steps for {num_stations} stations.")

# --- Calculate Metrics ---

# 1. Standard Metrics (Average per Station)
# multioutput='uniform_average' is the default for r2_score on 2D arrays
mae = mean_absolute_error(y_test_reshaped, y_pred_reshaped)
mse = mean_squared_error(y_test_reshaped, y_pred_reshaped)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_reshaped, y_pred_reshaped)

# Calculate MAPE
epsilon = 1e-10
mape = np.mean(np.abs((y_test_reshaped - y_pred_reshaped) / (y_test_reshaped + epsilon))) * 100

print("-" * 30)
print(f"Test MSE: {mse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAPE: {mape:.4f}%")
print(f"Test R2 (Avg per Station): {r2:.4f}")

# 2. Filtered R2 (Removing extremely poor stations)
# Matches the logic in test_full.ipynb
r2_per_station = r2_score(y_test_reshaped, y_pred_reshaped, multioutput='raw_values')
valid_stations = np.where(r2_per_station > -100)[0]
r2_filtered = r2_per_station[valid_stations].mean()

print(f"Test R2 (Filtered, > -100): {r2_filtered:.4f}")
print("-" * 30)

# --- Print Prediction Results (Sample) ---
print("\n--- Prediction Results (Sample) ---")
# Flatten again for DataFrame display
results_df = pd.DataFrame({
    'Station_Index': np.tile(np.arange(num_stations), complete_steps),
    'Actual': y_test_reshaped.flatten(),
    'Predicted': y_pred_reshaped.flatten()
})

# Map station index back to station name
if 'stations' in locals():
    results_df['Station_Name'] = results_df['Station_Index'].map(lambda x: stations[x])

print(results_df.head(10))

# Plot a sample station
if 'stations' in locals():
    sample_station = stations[0]
    station_data = results_df[results_df['Station_Name'] == sample_station]
    
    plt.figure(figsize=(15, 5))
    plt.plot(station_data['Actual'].values, label='Actual', alpha=0.7)
    plt.plot(station_data['Predicted'].values, label='Predicted', alpha=0.7)
    plt.title(f"Actual vs Predicted Footfall for {sample_station} (Test Set)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

Evaluated on 38 time steps for 436 stations.
------------------------------
Test MSE: 19495313.4640
Test MAE: 1858.5813
Test RMSE: 4415.3498
Test MAPE: 14.7818%
Test R2 (Avg per Station): -13609503697753932962988032.0000
Test R2 (Filtered, > -100): -1.1990
------------------------------

--- Prediction Results (Sample) ---
   Station_Index        Actual     Predicted
0              0   1631.000000   1391.550964
1              1  19198.857143  13806.509447
2              2   3226.571429   3744.072444
3              3   4798.166667   2225.960249
4              4  12011.571429  11938.328265
5              5  14976.714286  16234.401014
6              6  28038.714286  28299.662648
7              7   2066.857143   2013.683256
8              8   6612.000000   6778.793856
9              9   3740.857143   4112.520210
